In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple, List

## Data reading

In [ ]:
# incarcarea datelor

# data source: http://archive.ics.uci.edu/ml/datasets/Condition+Based+Maintenance+of+Naval+Propulsion+Plants
file = 'data.txt'
data_df = pd.read_csv(file, sep=r'\s+', header=None)
data = data_df.values

In [ ]:
# se sterge penultima coloana

data = # scrieti cod
print(data.shape)
assert data.shape == (11934, 17)

## Input processing

In [ ]:
# Functie pentru separarea datelor de intrare de cele de iesire

def get_input_output(mat: np.array) -> Tuple[np.array, np.array]:
    """Separa partea de intrare (variabile independente) de cea de iesire 
    (variabila dependenta).
    
    :param mat: np.array de forma (m, n). Fiecare linie este un caz 
    inregistrat, adica un vector de intrare. Primele n-1 coloane sunt valori 
    de intrare in model, ultima coloana este valoarea de iesire asociata 
    (ground truth).
    
    :return: un tuplu cu: matrice de (m, n-1) continand intrarile 
    (o linie = un vector de intrare); un vector coloana cu m linii continand
    valorile de iesire asociate intrarilor.
    """
    m, n = mat.shape
    X = # scrieti cod
    y = # scrieti cod
    assert np.alltrue(X == mat[:, :-1])
    assert np.alltrue(y[:, -1] == mat[:, -1])
    return X, y

In [ ]:
# separa matricea cu vectorii de intrare si vectorul coloana cu iesirile asociate

X, y = # scrieti cod

In [ ]:
# Functie pentru scalarea datelor in intervalul [0, 1]. 
# Scalele se calculeaza pe baza datelor furnizate

def scale_matrix(mat: np.array) -> np.array:
    """Preia o matrice cu valori reale si scaleaza toate coloanele in 
    intervalul [0, 1]. Coloanele constante vor fi transformate in 1.
    
    :param mat: matrice de forma (m, n)
    :return: matrice cu valori scalate in [0, 1]
    """
    
    # calculeaza pe ce indecsi minimele si maximele difera. 
    # Doar pe acestea se va face scalare. Coloanele pentru care minimele si 
    # maximele sunt egale au valori constante si vor fi umplute automat cu 1.
    min_cols, max_cols = # scrieti codpentru a determina minimele si maximele 
    # de pe fiecare coloana a lui mat
    different_min_max = # scrieti cod pentru a determina pe ce coloane 
    # minimele si maximele difera
    result = np.ones_like(mat)
    result[:, different_min_max] = # scrieti cod pentru scalarea datelor folosind min_cols, max_cols
    assert result.shape == mat.shape
    assert np.alltrue(np.min(result, axis=0) >= 0)
    assert np.alltrue(np.max(result, axis=0) <= 1)
    return result, min_cols, max_cols

In [ ]:
# Functie pentru scalarea datelor. 
# Scalele sunt furnizate, de regula calculate pe baza datelor din setul 
# de antrenare

def scale_matrix_with_known_minmax(mat: np.array, min_cols: np.array, max_cols: np.array) -> np.array:
    """Preia o matrice  :param mat: si efectueaza scalarea valorilor in 
    intervalul [0, 1] folosind valori de min si max pe fiecare coloana 
    precalculate si date prin parametrii :param min_cols:, :param max_cols:.
    
    :param mat: matricea de (m, n) care trebuie scalata.
    :param min_cols: minime pe coloane, precalculate
    :param max_cols: maxime pe coloane, precalculate
    :return: matrice de (m, n) cu valori scalate.
    """
    # calculeaza pe ce indecsi minimele si maximele difera. 
    # Doar pe acestea se va face scalare. Coloanele pentru care minimele si 
    # maximele sunt egale au valori constante si vor fi umplute automat cu 1.
    different_min_max = # scrieti cod pentru a determina pe ce coloane difera 
    # minimele si maximele (coloane cu valori necosntante)
    result = np.ones_like(mat)
    result[:, different_min_max] = # scrieti cod care sa scaleze datele 
    # folosind parametrii min_cols si max_cols dati
    assert result.shape == mat.shape
    return result

In [ ]:
# scaleaza matricele

X, min_cols, max_cols = # scrieti cod pentru a scala X intre 0 si 1 # aici min_cols si max_cols nu vor fi de fapt folosite
y, _, _ = # scrieti cod pentru a scala coloana y in [0, 1]

In [ ]:
# calculeaza matricea de design: coloana de 1 adaugata in fata datelor originare

def design_matrix(mat: np.array) -> np.array:
    """Preia o matrice de forma (m, n) si returneaza o matrice de forma 
    (m, n+1), in care prima coloana este 1, iar ultimele n coloane sunt 
    coloanele lui :param mat:.
    
    :param mat: matrice de forma (m, n)
    :return: matrice de forma (m, n+1), cu prima coloana plina cu 1, ultimele 
    n coloane sunt cele din :param mat:
    """
    m, n = mat.shape
    result = # scrieti cod
    assert result.shape == (m, n+1)
    assert np.alltrue(result[:, 0] == 1)
    assert np.alltrue(result[:, 1:] == mat)
    return result

In [ ]:
# calculeaza matricea de design: vechea matrice la care se adauga pe prima
# pozitie coloana cu 1
X = # scrieti cod

## Linear regression with stochastic gradient descent

### Model implementation

In [ ]:
# Functie care implementeaza modelul de regresie liniara

def h(X: np.array, w: np.array) -> np.array:
    """Pentru un model liniar specificat prin parametrii w si un set de date 
    de intrare X produce valorile estimate de model: 
    y_hat[i] = X[i, 0] * w[0] + X[i, 1] * w[1] + ...  
   
   :param X: valori de intrare, structurate ca np.array de forma (m, n)
   :param w: vector de ponderi, de forma (n, 1)
   :return: vector cu valori de iesire estimate, de forma (n, 1). 
    """
    assert w.shape == (X.shape[1], 1)
    assert X.shape[1] == w.shape[0]
    y_hat = # scrieti cod; pentru punctaj maxim, vectorizat
    assert y_hat.shape == (X.shape[0], 1)
    return y_hat

### MSE function

In [ ]:
# Implementare de eroare patratica medie

def mean_squared_error(y: np.array, y_hat: np.array) -> float:
    """Functie de eroare patratica medie.
    
    :param y: vector coloana de m elemente, continand valorile de iesire reale 
    (ground truth)
    :param y_hat': vector coloana de m elemente, continand valorile de iesire
    estimate.
    :return: eroarea patratica medie.
    """
    result = # scrieti cod
    return result

### Cost function

In [ ]:
# Functia de eroare

def J(X:np.array, w:np.array, y:np.array, lmbda: float = 0) -> float:
    """Calculeaza valoarea functiei de eroare pentru intrarea curenta :param X:, 
    estimarea produsa de model folosind ponderile :param w: si iesirea corecta
    :param y: (ground truth).
    
    :param X: matrice de valori de intrare pentru care modelul de regresie 
    liniara produce estimari. Are forma (m, n)
    :param w: vector coloana de ponderi, definind coeficientii modelului de 
    regresie liniara. Are forma (n, 1).
    :param y: vector coloana de valori de iesire cunoscute (ground truth). 
    Are forma (n, 1).
    :param lmda: coeficientul de penalizare L2
    :return: jumatate din mean squared error plus penalizarea L2
    """
    assert w.shape == (X.shape[1], 1)
    assert y.shape == (X.shape[0], 1)
    assert lmbda >= 0
    m = X.shape[0]
    y_hat = # scrieti cod
    assert y_hat.shape == (X.shape[0], 1)
    term_squared_error = # scrieti cod
    l2_error = # scrieti cod
    return term_squared_error + l2_error 

### Stochastic gradient descent implementation

In [ ]:
# Calcul de gradient. gradientul se va folosi in actualizarea ponderilor. 

def gradient(X:np.array, w:np.array, y:np.array, lmbda: float = 0) -> np.array:
    """Calculeaza gradientul functiei de eroare
    
    :param X: matrice de valori de intrare pentru care modelul de regresie 
    liniara produce estimari. Are forma (m, n)
    :param w: vector coloana de ponderi, definind coeficientii modelului de 
    regresie liniara. Are forma (n, 1).
    :param y: vector coloana de valori de iesire cunoscute (ground truth). 
    Are forma (n, 1).
    :param lmda: coeficientul de penalizare L2
    :return: vector gradient
    """
    assert w.shape == (X.shape[1], 1)
    assert y.shape == (X.shape[0], 1)
    assert lmbda >= 0
    m, _ = X.shape
    y_hat = # scrieti cod
    grad = # scrieti cod vectorizat, daca vreti sa rulati in timp rezonabil :)
    grad[1:, ] += # scrieti cod pentru a include si gradientul pentru termenul L2
    assert grad.shape == w.shape
    return grad

In [ ]:
# Implementarea algoritmului de antrenare

def train(X:np.array, y:np.array, max_iters:int = 100000, alpha:float = 0.01, lmbda: float = 0.01) -> Tuple[np.array, List[float]]:
    """Aplica algoritmul de instruire cu gradient. 
    
    :param X: matrice de valori de intrare pentru care modelul de regresie 
    liniara produce estimari. Are forma (m, n)
    :param y: vector coloana de valori de iesire cunoscute (ground truth). 
    Are forma (n, 1).
    :param alpha: rata de invatare
    :param lmda: coeficientul de penalizare L2
    :return: tuplu continand ponderile modelului rezultate dupa instruire, lista de erori pe setul de antrenare
    """
    m, n = X.shape
    assert y.shape == (m, 1)
    assert X.shape[0] == y.shape[0]
    w = # scrieti cod pentru initializare ponderi
    losses = [None] * max_iters
    for i in range(max_iters):
        # calculeaza si adauga loss-ul in lista
        # calculeaza gradientul si actualizaeaza ponderile w
        loss = # scrieti cod
        losses[i] = loss
        grad = # scrieti cod
        w -= # scrieti cod pentru actualizare de ponderi folosind gradientul
    return w, losses

In [ ]:
# calcul ponderi si valorile functiei de cost

w, losses = train(X, y, max_iters=100000, alpha=0.3, lmbda=0.1)

In [ ]:
# desenarea curbei de cost pe setul de antrenare

plt.figure(figsize=(12, 8))
plt.xlabel('Epoca')
plt.ylabel('Valoare de eroare')
plt.plot(losses)

In [ ]:
# Obtine valorile estimate de model, calculeaza eroarea intre estimare si 
# ground truth.

# estimare facuta de model
y_hat = # scrieti cod
# mean squared error pe setul de antrenare
error_sgd = # scrieti cod

print('Eroarea determinata prin algoritmul stochastic gradient descent:', error_sgd)
print('Vectorul de ponderi:', w[:, 0])

### Pseudoinverse method

In [ ]:
# se calculeaza w cu metoda pseudoinversei 
w_pinv = # scrieti cod pentru calcul folosinf metoda pinv din numpy

# se calculeaza estimarea data cu ponderile w_pinv
y_hat_pinv = # scrieti cod

# se calculeaza eroarea patratica medie
error_pinv= # scrieti cod

print('Eroarea determinata prin metoda pseudoinversei:', error_pinv)

In [ ]:
# print(np.linalg.norm(gradient(X, w_pinv, y)))

In [ ]:
# afiseaza ponderile determinate prin metoda pseudoinversei

print(w_pinv[:, 0])

## Split validation

In [ ]:
# imparte setul initial in 70% antrenare si 30% testare

data = # scrieti cod pentru a face o permutare aleatoare
split_percentage = 0.7
split_index = # scrieti cod; care e linia la care se opresete setul de train
data_train = # scrieti cod
data_test = # scrieti cod

X_train, y_train = # scrieti cod pentru a separa intrarea de iesire
X_test, y_test = # scrieti cod pentru a separa intrarea de iesire

In [ ]:
# scaleaza datele de intrare din setul de antrenare si obtine minimele 
# si maximele pe coloane
X_train, min_X_train, max_X_train = # scrieti cod 
# scaleaza datele de iesire din setul de antrenare si obtine minimul si 
# maximul pe coloana
y_train, min_y_train, max_y_train = # scrieti cod 

In [ ]:
# scaleaza datele de intrare din setul de test, folosind minimele si maximele 
# pe coloane aflate de la X_train
# se foloseste functia scale_matrix_with_known_minmax
X_test = # scrieti cod 

# scaleaza datele de iesire din setul de test, folosind minimul si maximul 
# pe coloana aflate de la y_train
y_test = # scrieti cod 

In [ ]:
# antreneaza linear regression pe setul de antrenare, obtine ponderile
# modelului

w_train, losses_train = # scrieti cod pentru antrenare pe X_train, y_train 
# restul parametrilor ca la apelul de antrenare de mai sus

In [ ]:
# obtine estimarile date de model pe setul de testare

y_hat_test = # scrieti cod 
print('MSE pentru setul de testare:', mean_squared_error(y_test, y_hat_test))

In [ ]:
# reprezentarea diferentei intre ground truth si estimare pe setul de testare
plt.figure(figsize=(12, 8))
plt.title('Erori reziduale pe setul de testare')
plt.plot(y_test - y_hat_test)
plt.xlabel('Data de intrare')
plt.ylabel('Diferenta intre ground truth si valoarea estimata')